In [7]:
from supabase import create_client,Client
import os
# get environment variables from .env file
import os

def load_env():
    with open('.env', 'r') as f:
        for line in f:
            if line.strip():
                key, value = line.strip().split('=', 1)
                os.environ[key] = value

load_env()


# get environment variables
url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_KEY")
supabase = create_client(url, key)



In [11]:
# functions to load data
import json
import requests

def get_translations(count):
    url = 'https://read.84000-translate.org/translations.json?api-version=0.4.0'
    response = requests.get(url)
    works = json.loads(response.text)
    # only return the required number of translations
    return works['work'][:count]

def get_work(Work_id):
    work = json.loads(f"""../../data-json/{Work_id}.json""")
    return work

def get_titlte(work,WorkId):
    titles = work['work'][0]['title']
    titles_dicts = [dict({'work_uuid': WorkId,
                          'content': title['content'],
                          'titleType': title['titleType'],
                          'language': title['language'],
                          'titleMigrationId': title['titleMigrationId']}) for title in titles]
    return titles_dicts

def get_passages(work,WorkId):
    passages = work['translation']['passage']
    passage_dict = [dict({'work_uuid': WorkId,
                          'content': passage['content'],
                          'segmentationType': passage['segmentationType'],
                          'xmlId': passage['xmlId'],
                          'parentId': passage['parentId'],
                          'passageLabel': passage['passageLabel'],
                          'passageSort': passage['passageSort']}) for passage in passages]
    return passage_dict


In [30]:
# upsert into supabase
def store_work(Current_Work,translation):
    work_dict = dict({'xmlId': Current_Work['xmlId'],
                       'url': Current_Work['url'],
                       'type': Current_Work['workType'],
                       'toh': Current_Work['toh'],
                       'title': Current_Work['title'],
                      'migrationJson': translation})
    try:
        work_data,count = supabase.table('works').insert(work_dict).execute()
        return work_data[1][0]['uuid']
    except Exception as e:
        print ("Error Occured",e)
        return None
    
    
def store_titles(titles_dicts):
    try:
        title_data,count = supabase.table('titles').upsert(titles_dicts).execute()
        print("Added",len(title_data[1]))
    except Exception as e:
        print("Error Occured",e)
        # If error occurs delete all just inserted data
        try:
            if (title_data[1]):
                for inserts in title_data[1]:
                    print(inserts)
                    data_temp,count_temp = supabase.table('titles').delete().eq('id',inserts['id']).execute()
                    print('deleted',data_temp[1])
        except IndexError:
            print("No Data to Delete")
            
def store_passages(passage_dict):
    try:
        passage_data,count = supabase.table('segments').upsert(passage_dict).execute()
        print("Added",len(passage_data[1]))
    except Exception as e:
        print("Error Occured",e)
        # If error occurs delete all just inserted data
        try:
            if (passage_data[1]):
                for inserts in passage_data[1]:
                    print(inserts)
                    data_temp,count_temp = supabase.table('passages').delete().eq('id',inserts['id']).execute()
                    print('deleted',data_temp[1])
        except IndexError:
            print("No Data to Delete")


Added 16


In [6]:
# Excuter 
# 1. get required  number of translations
# 2. Load the data into Supabase and get back the workId
# 3. Get the work data from the local file
# 4. Get the titles and passages
# 5. Create Title and passage dictionary with WorkId

def load_data(translation_count):
    works = get_translations(translation_count)
    for work in works['work']:
        translation = get_work(WorkId)
        titles_dicts = get_titlte(translation,WorkId)
        passage_dict = get_passages(translation,WorkId)
        
        if ( translation && titles_dicts && passage_dict):
            WorkId = store_work(work,translation)
            if (WorkId):
                store_titles(titles_dicts)
                store_passages(passage_dict)
    
    return


In [ ]:
load_data(1)

In [13]:
display(get_translations(1))

[{'workId': 'UT22084-001-001',
  'workType': 'eft:translation',
  'url': '/translation/UT22084-001-001.json?api-version=0.4.0&annotate=true',
  'htmlUrl': 'https://read.84000.co/translation/UT22084-001-001.html',
  'catalogueWorkIds': 'toh1-1'}]